In [ ]:
import os
import logging
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split, GridSearchCV

# 配置日志记录器，设置日志级别和格式
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_and_preprocess_data(file_path):
    """
    加载并预处理数据。
    """
    try:
        # 使用pandas读取CSV文件
        data = pd.read_csv(file_path)
        
        # 删除不相关的特征
        data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
        
        # 对分类特征进行独热编码
        data = pd.get_dummies(data, columns=['Geography', 'Gender'], drop_first=True)
        
        # 数据筛选，保留年龄小于60且信用评分大于400的记录
        data = data[(data['Age'] < 60) & (data['CreditScore'] > 400)]
        
        # 记录日志，表示数据加载和预处理成功
        logging.info("Data loaded and preprocessed successfully.")
        return data
    
    except Exception as e:
        # 如果发生异常，记录错误日志并抛出异常
        logging.error(f"Error loading data: {e}")
        raise

def train_decision_tree(X, y, criterion, params=None):
    """
    使用指定的标准和可选参数训练决策树模型。
    """
    # 初始化决策树分类器
    dtree_model = DecisionTreeClassifier(criterion=criterion, random_state=42)
    
    if params:
        # 如果提供了参数，则使用网格搜索来寻找最佳参数
        grid_search = GridSearchCV(dtree_model, params, cv=5, scoring='accuracy')
        grid_search.fit(X, y)
        # 记录最佳参数
        logging.info(f"Best parameters for {criterion.upper()} Decision Tree: {grid_search.best_params_}")
        return grid_search.best_estimator_
    
    else:
        # 如果没有提供参数，直接训练模型
        dtree_model.fit(X, y)
        logging.info(f"{criterion.upper()} Decision Tree trained.")
        return dtree_model

def export_decision_tree(model, X, output_path):
    """
    将决策树导出为DOT格式。
    """
    # 使用export_graphviz导出决策树
    export_graphviz(model, out_file=output_path, feature_names=X.columns,
                    class_names=['Not Exited', 'Exited'], filled=True, rounded=True)
    # 记录日志，表示决策树已导出
    logging.info(f"Decision tree exported to {output_path}.")

def main():
    # 定义文件路径和输出目录
    file_path = r"C:\Users\14421\Desktop\银行客户流失\代码及数据\Churn_Modelling.csv"
    output_dir = r'C:\Users\14421\Desktop\Churn_Modelling'

    # 如果输出目录不存在，则创建它
    os.makedirs(output_dir, exist_ok=True)

    # 加载并预处理数据
    data = load_and_preprocess_data(file_path)
    # 分离特征和目标变量
    X = data.drop('Exited', axis=1)
    y = data['Exited']

    # 划分数据集为训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # 定义超参数范围
    params = {
        'max_depth': [3, 5, 7, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 5]
    }

    # 训练并评估ID3决策树
    id3_tree = train_decision_tree(X_train, y_train, criterion='entropy', params=params)
    export_decision_tree(id3_tree, X, os.path.join(output_dir, "id3_tree.dot"))

    # 训练并评估C4.5决策树
    c45_tree = train_decision_tree(X_train, y_train, criterion='entropy', params=params)
    export_decision_tree(c45_tree, X, os.path.join(output_dir, "c45_tree.dot"))

    # 训练并评估Gini决策树
    gini_tree = train_decision_tree(X_train, y_train, criterion='gini', params=params)
    export_decision_tree(gini_tree, X, os.path.join(output_dir, "gini_tree.dot"))

    # 记录日志，表示所有决策树模型已训练并导出
    logging.info("All decision tree models trained and exported successfully.")

if __name__ == "__main__":
    main()
